In [1]:
import pickle

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm

In [2]:
import warnings
warnings.simplefilter("ignore", UserWarning)

In [3]:
IN_DIR = "D:\\result\\20221027\\"
OUT_DIR = "D:\\result\\20221027\\"
sub_dir_list = "cbi_q_10\\1\\"

In [4]:
time_table = pd.read_csv("time.csv", header=None, index_col=0)
time_table = time_table.values[0, :34].astype(np.int)

/Users/loblium/opt/anaconda3/envs/venv/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  


In [5]:
ptv_result = np.zeros((1, 4))

In [6]:
for t in range(1, 11):
    for i in tqdm(range(t, 10900, 10)):
        for j in range(17):
            if time_table[2 * j] <= i < time_table[2 * j + 1]:
                try:
                    r = np.loadtxt(OUT_DIR + "ptv\\" + sub_dir_list + f"pp_{i}.csv", delimiter=',', skiprows=1)
                except StopIteration:
                    r = None

                if r is None:
                    pass
                elif r.ndim == 1:
                    if len(r) > 0:
                        r = r[np.newaxis, :]
                        ptv_result = np.vstack([ptv_result, r])

                elif r.ndim == 2:
                    ptv_result = np.vstack([ptv_result, r])

  2%|▏         | 20/1090 [00:00<00:00, 20530.12it/s]


OSError: D:\result\20221027\ptv\cbi_q_10\1\pp_201.csv not found.

In [ ]:
ptv_result = pd.DataFrame(ptv_result, columns=['x', 'y', 'dx', 'dy'])
for row in ptv_result.index:
    if (ptv_result.loc[row] == 0).all():
        ptv_result.drop(row, axis=0, inplace=True)

In [ ]:
ptv_result['x'] = ptv_result['x'] - 498
ptv_result['y'] = 51 - ptv_result['y']
ptv_result['dy'] = - ptv_result['dy']

In [ ]:
ptv_result['r'] = np.sqrt(ptv_result['x']**2 + ptv_result['y']**2)
ptv_result['theta'] = np.arctan2(ptv_result['y'], ptv_result['x'])
ptv_result['dr'] = ptv_result['dx'] * np.cos(ptv_result['theta']) + ptv_result['dx'] * np.sin(ptv_result['theta'])

In [ ]:
fig, ax = plt.subplots(figsize=(20, 12))
plt.xlim(-512, 512)
plt.ylim(-500, 100)
c = ax.quiver(ptv_result['x'], ptv_result['y'], ptv_result['dx'], ptv_result['dy'], ptv_result['dr'], cmap="jet")
fig.colorbar(c)
plt.show()

In [ ]:
# with open(f'ptv_cbi_q_10.pickle', mode='wb') as fp:
#     pickle.dump(ptv_result, fp)